In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.instance.SAT_Instance import SAT_Instance, SAT_from_index_file
from src.solver.SAT_Riss_Solver import SAT_Riss_Solver
from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import get_model_training_data

In [2]:
db = DB("database/run-30-sur-50-1027449.db")
X, y = get_model_training_data(db)

In [5]:
query = f"""
select 
    {db.SCHEMA.EVALUATIONS}.cost,
    {db.SCHEMA.SOLVERS}.*,
    {db.SCHEMA.INSTANCES}.*
from {db.SCHEMA.EVALUATIONS}
join {db.SCHEMA.INSTANCES} on {db.SCHEMA.EVALUATIONS}.instance_id = {db.SCHEMA.INSTANCES}.id
join {db.SCHEMA.SOLVERS} on {db.SCHEMA.EVALUATIONS}.solver_id = {db.SCHEMA.SOLVERS}.id
"""
df = db.query2df(query)
df = df.drop(columns=["id", "filepath"])
df = df.dropna()

In [20]:
df.columns

Index(['cost', 'K', 'R', 'act_based', 'actDec', 'actIncMode', 'actStart',
       'alluiphack', 'biAsserting', 'ccmin_mode',
       ...
       'CG_coeff_variation', 'CG_min', 'CG_max', 'CG_entropy',
       'cluster_coeff_mean', 'cluster_coeff_coeff_variation',
       'cluster_coeff_min', 'cluster_coeff_max', 'cluster_coeff_entropy',
       'CG_featuretime'],
      dtype='object', length=158)

In [7]:
instances = SAT_from_index_file(
    filepath=DATA_DIR / "SAT" / "index.json",
    max_cost=100.0,
    max_time=10.0,
)

train_instances = instances[:30]
test_instances = instances[30:80]

In [10]:
instance = train_instances[0]
instance.calculate_features()
len(instance.features)

54

In [12]:
solver = SAT_Riss_Solver()
solver

Solver(id=386181383168834235)

In [14]:
instance.get_array().shape

(54,)

In [15]:
solver.get_array().shape

(101,)

In [16]:
X = np.concatenate([solver.get_array(), instance.get_array()])
X = X.reshape(1, -1)

In [32]:
keys_solver = list(solver.config.keys())
keys_solver = [k.replace("-", "_").replace("+", "_") for k in keys_solver]

keys_instance = list(instance.features.keys())
keys_instance = [k.replace("-", "_").replace("+", "_") for k in keys_instance]

In [33]:
df.columns.difference(set(keys_solver + keys_instance))

Index(['cost', 'max_cost', 'max_time'], dtype='object')

In [40]:
df["max_time"]

0       10.0
1       10.0
2       10.0
3       10.0
4       10.0
        ... 
2216    10.0
2217    10.0
2218    10.0
2219    10.0
2220    10.0
Name: max_time, Length: 2221, dtype: float64

In [41]:
# index = []
# for filepath in list((DATA_DIR / "SAT").glob("*.cnf")):
#     v = "/".join(filepath.parts[-2:])
#     index.append(v)

# index

In [39]:
# for filepath in list((DATA_DIR / "SAT").glob("*.cnf")):
#     # Read the file
#     with open(filepath, 'r') as f:
#         lines = f.readlines()
    
#     # Remove last 4 lines and add one empty line
#     modified_lines = lines[:-4] + ['\n']
    
#     # Write back to the file
#     with open(filepath, 'w') as f:
#         f.writelines(modified_lines)
    
#     print(f"Processed: {filepath.name}")

In [8]:
solver = SAT_Riss_Solver()
instances = SAT_from_index_file(filepath=DATA_DIR / "SAT" / "index_u150.json", max_cost=10.0, max_time=10.0)
